### Simulation Experiments Evaluation

A revised environment was set to validate that the agent can adapt its behavior appropriately in new environments based on the learned reward function.

In [10]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from grid_world.grid_world import GridWorld
from DMEIRL.DeepMEIRL_FC import DeepMEIRL_FC
from grid_world.trajGen_grid_world import GridWorld_trajGen
from utils import utils

from grid_world.data_parser import DataParser
from grid_world.grid_world import GridWorld
import torch

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Reading environmental images

In [ ]:
parser = DataParser(width=10,height=10)
from PIL import Image
import os
from tqdm import tqdm
tracks_path_train = 'demo_label/train/'
tracks_path_eval = 'demo_label/eval/'
file_names_train = os.listdir(tracks_path_train)
file_names_eval = os.listdir(tracks_path_eval)
#read images
imgs_train = []
imgs_eval = []
for file_name in file_names_train:
    imgs_train.append(Image.open(tracks_path_train + "/" + file_name))
for file_name in file_names_train:
    imgs_eval.append(Image.open(tracks_path_eval + "/" + file_name))
#parse images from eval 
# for i in tqdm(range(len(imgs_train))):
#     parser.ParseEnvironmentFromImage(imgs_train[i],file_names_train[i].split('.')[0],save_path='demo_env&feature')
for i in tqdm(range(len(imgs_eval))):
    parser.ParseEnvironmentFromImage(imgs_eval[i],file_names_eval[i].split('.')[0],save_path='demo_env&feature')

100%|██████████| 4/4 [00:00<00:00, 139.61it/s]


In [20]:
model = DeepMEIRL_FC(n_input=4,layers=(16,32,64,32,16))
model.to('cuda')
model.load_state_dict(torch.load('tanh_model.pth'))
model.eval()
model.cuda()


AssertionError: Torch not compiled with CUDA enabled

# Figure 9

figure 9 (a): Labeling of new adjusted environmental elements were collected in "\demo_dmeirl\demo_label\eval" to validate that the agent can adapt its behavior appropriately in new environments based on the learned reward function.

figure 9 (b): The outcome of trajectory prediction in the new environment matched expectations.


In [29]:
n = np.nan
state_mat = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, n, n, n, n],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],])

world_trajGen = GridWorld_trajGen(10,10,states_matrix=state_mat,
                                  features_folderPath='demo_env&feature/features_grid/0312_10x10',
                                  rewards_mul=[1,-2,-1,-1],
                                  init_states=list(range(0,10)),
                                  trans_prob=0.6,discount=0.95,
                                  model = model)

world_trajGen.ShowLearnedRewards()

In [32]:
world_trajGen.GenerateTrajectoriesWithLearnedReward(traj_count=200,traj_length=30)

100%|██████████| 200/200 [00:00<00:00, 1560.38it/s]


,m,trajs
0,1,"[(1, 1, 1), (1, 1, 11), (11, 1, 21), (21, 4, 2..."
1,2,"[(4, 1, 3), (3, 1, 13), (13, 3, 12), (12, 1, 2..."
2,3,"[(7, 3, 7), (7, 3, 7), (7, 3, 6), (6, 3, 5), (..."
3,4,"[(4, 1, 14), (14, 3, 13), (13, 3, 12), (12, 1,..."
4,5,"[(5, 3, 5), (5, 3, 4), (4, 1, 14), (14, 3, 13)..."
...,...,...
195,196,"[(3, 1, 13), (13, 3, 23), (23, 3, 23), (23, 3,..."
196,197,"[(1, 1, 11), (11, 1, 21), (21, 4, 22), (22, 0,..."
197,198,"[(7, 3, 6), (6, 3, 5), (5, 3, 6), (6, 3, 16), ..."
198,199,"[(4, 1, 14), (14, 3, 13), (13, 3, 12), (12, 1,..."


In [33]:
world_trajGen.ShowTrajs_Learner()